# Segmenting and Clustering Neighborhoods in Toronto

_The first task requires to scrape and parse the given web-page. I'm going to use BeautifulSoup4 and requests libraries for this purpose._

In [5]:
# using requests load given web-page into object page 
import requests
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page

<Response [200]>

_Also I need pandas for loading parsed data into dataframe_

In [6]:
import pandas as pd

_Let's install BeautifulSoup4_

In [4]:
!pip install beautifulsoup4

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [34]:
!conda install -c anaconda beautifulsoup4

Solving environment: done

# All requested packages already installed.



In [7]:
#here we parse page content
#page.content
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [1]:
#print(soup.prettify())

_Let's create dataframe for our data. It consists of 3 columns PostalCode, Borough, Neighborhood. And than load data into it from object soup_ 

In [10]:
#Let's create dataframe for our data. It consists of 3 columns  
CACodeDf = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

In [19]:
#And finaly, let's load data from our page into dataframe  CACodeDf
i = 0
for tr in soup.find_all('tr',text=True)[1:]:
    tds = tr.find_all('td')  
    my_str = tds[2].text.replace('\n', '')
    CACodeDf.loc[i] = [tds[0].text]+ [tds[1].text] + [my_str]
    i = i+1
print      ("done")


done


_Here is the result. First 10 rows of our dataset_

In [28]:
CACodeDf.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
9,M8A,Not assigned,Not assigned


In [29]:
CACodeDf.shape

(288, 3)

_Now I have to process the data accordingly instructions given into assignment. First, I should remove all rows where Borough is Not assigned_

In [47]:
#Now , since we have dataframe filled with all necessary data, we should clear it from inappropriate rows
CACodeDf = CACodeDf[CACodeDf.Borough != "Not assigned"]
#CACodeDf.reset_index(inplace=True,)
CACodeDf.drop('level_0',axis=1, inplace=True)
CACodeDf.head(10) 

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [31]:
CACodeDf.shape

(211, 3)

_Now, let's update values of the cells where  Neighborhood is Not assigned. The new value of Neighborhood  for such rows must be equal its Borough value_

In [48]:
CACodeDf.loc[CACodeDf['Neighborhood'] == "Not assigned", ['Neighborhood']] = CACodeDf['Borough']
CACodeDf.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [38]:
CACodeDf.shape

(211, 3)

_Finally, I have to transform our dataframe into the new one, so Postal Code would be unique_

In [49]:
df = CACodeDf[['PostalCode','Borough']]
#df.drop_duplicates()
df=df.drop_duplicates(['PostalCode','Borough'])[['PostalCode','Borough']]
df['Neighborhood'] = df.apply(lambda row: "", axis=1)
df.head(10)
#df.shape

,PostalCode,Borough,Neighborhood
0,M3A,North York,
1,M4A,North York,
2,M5A,Downtown Toronto,
4,M6A,North York,
6,M7A,Queen's Park,
7,M9A,Etobicoke,
8,M1B,Scarborough,
10,M3B,North York,
11,M4B,East York,
13,M5B,Downtown Toronto,


In [50]:
for index, row in df.iterrows():
    my_str = ''
    for index2, row2 in CACodeDf.iterrows():
        if row2['PostalCode'] == row['PostalCode'] and row2['Borough'] == row['Borough']:
            my_str = row2['Neighborhood']+","+my_str
    row['Neighborhood'] = my_str[:-1]
    
df.head(10)     

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
4,M6A,North York,"Lawrence Manor,Lawrence Heights"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Malvern,Rouge"
10,M3B,North York,Don Mills North
11,M4B,East York,"Parkview Hill,Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District,Ryerson"


In [42]:
df.shape

(103, 3)

_So, you can see the final dataset consists of 103 rows_